## Read the data

We will download the MNIST dataset for training a classifier. Torch provides a convenient function for that.

The MNIST dataset is composed of images of digits that must be classified with labels from 0 to 9. The inputs are 28x28 matrices containing the grayscale intensity in each pixel.

In [3]:
import numpy as np
import mnist

train_x = mnist.train_images()
train_y = mnist.train_labels()
test_x = mnist.test_images()
test_y = mnist.test_labels()

print('%d training instances and %d test instances' % (len(train_x), len(test_x)))

60000 training instances and 10000 test instances


Check the shape of our training data to see how many input features there are:

In [4]:
print(train_x.shape)
print(train_x[0])

(60000, 28, 28)
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 25

### Formatting

Each sample is a 28x28 matrix. But we want to represent them as vectors, since our model doesn't take any advantage of the 2-d nature of the data.

So, we reshape the data:

In [7]:
num_features = 28 * 28
new_shape = [60000, num_features]
train_x_vectors = train_x.reshape(new_shape)
print(train_x_vectors.shape)
print(train_x_vectors[0])

(60000, 784)
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   3  18  18  18 126 136 175  26 166 255
 247 127   0   0   0   0   0   0   0   0   0   0   0   0  30  36  94 154
 170 253 253 253 253 253 225 172 253 242 195  64   0   0   0   0   0   0
   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251  93  82
  82  56  39   0   0   0   0   0   0   0   0   0   0   0   0  18 219 253
 253 253 253 253 198 182 247 241   0  

When we reshape an array (or torch tensor, for that matter), we don't need to specify all dimensions. We can leave one as -1, and it will be automatically determined from the size of the data. This is useful when we don't know a priori the shape of some array.

In [8]:
train_x_vectors = train_x.reshape([-1, num_features])
test_x_vectors = test_x.reshape([-1, num_features])

Also, the values are integers in the range $[0, 255]$. It is better to work with float values in a smaller interval, such as $[0, 1]$ or $[-1, 1]$. There are some more elaborate normalization techniques, but for now let's just normalize it to $[0, 1]$.

In [9]:
train_x_vectors /= 255
test_x_vectors /= 255

TypeError: No loop matching the specified signature and casting
was found for ufunc true_divide

Oops! Notice that the arrays had integer values, but the result of the division would be floats. The `dtype` of the arrays cannot be changed by arithmetic operators; we need instead to create new arrays.

Keep in mind that data type are a common source of errors!

In [18]:
train_x_norm = train_x_vectors / 255
test_x_norm = test_x_vectors / 255
print(train_x_norm[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

Now, check the labels:

In [19]:
print(np.unique(train_y))
num_classes = len(np.unique(train_y))

[0 1 2 3 4 5 6 7 8 9]


### Creating a simple linear classifier

Our input has 128 dimensions (each one is a pixel), and the output has 26 possible classes. We will create a weight matrix $w$ and a bias vector $b$.

The parameter `requires_grad` tells pytorch that their values are adjustable through gradient backpropagation.

In [20]:
w = torch.randn([num_features, num_classes], requires_grad=True)
b = torch.randn([num_classes], requires_grad=True)

For illustration purposes, let's take the first row of the data and create a pytorch tensor with it.

In [61]:
x0 = torch.tensor(train_x_norm[0])
torch.matmul(x0, w) + b

RuntimeError: Expected object of type torch.DoubleTensor but found type torch.FloatTensor for argument #2 'mat2'

Again, take care with data types! The inputs were double precision floats (64 bit) and the weights are normal floats (32 bits). Let's explicitly create the batch tensor with normal floats.

In [64]:
x0 = torch.tensor(train_x_norm[0], dtype=torch.float)
logits = torch.matmul(x0, w) + b

This is how the logits look like. Think of them as the scores for each instance/class combination.

In [114]:
logits

tensor([-12.6724,   7.6523,   2.0795,  -6.4135,   0.1790,  -1.4879,   9.7279,
          0.3401,  -4.0881,  -5.3371], grad_fn=<ThAddBackward>)

We want to take the highest scoring class for each instance, i.e., the argmax:

In [118]:
answer = torch.argmax(logits)
answer

tensor(6)

What are the correct classes for those? Most of them must be wrong, we just initialized weights randomly.

In [112]:
label = torch.tensor(train_y[0], dtype=torch.long)
label

tensor(5)

#### Loss

We can compute the loss as the mean cross-entropy, as usual for classification problems. Remember that the cross-entropy between the true label distribution $p$ and the predicted $q$ is computed as:

\begin{align}
loss(x, y, \theta) = -\sum_c p(y=c|x) \log q(y=c|x, \theta)
\end{align}

for every label $c$.

The true distribution $p$ is one for the correct label and 0 elsewhere; the predicted $q$ can be computed as the softmax over the logits.

In [106]:
p = torch.zeros([num_classes])
p[label] = 1
p

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [107]:
q = torch.softmax(logits, dim=0)
q

tensor([1.6599e-10, 1.1143e-01, 4.2343e-04, 8.6761e-08, 6.3295e-05, 1.1953e-05,
        8.8800e-01, 7.4362e-05, 8.8755e-07, 2.5454e-07],
       grad_fn=<SoftmaxBackward>)

In [108]:
cross_entropy = -torch.sum(p * torch.log(q))
cross_entropy

tensor(11.3345, grad_fn=<NegBackward>)

We can also use pytorch's own function for that! 

We just need to reshape our logits to be a 1x10 tensor and the label to be a 1-dim tensor. This is because usually we process lots of inputs at once.

In [119]:
logits = logits.reshape([1, -1])
label = label.reshape([1])

loss = torch.nn.functional.cross_entropy(logits, label)
loss

tensor(11.3345, grad_fn=<NllLossBackward>)

#### Gradients

Now we have to compute the gradients to ajust weights. If we take the partial derivative of the cross-entropy (formula above!) with respect to weights $w_c$, we eventually end up with:

\begin{align}
\frac{\partial loss(x, y, \theta)}{\partial w_c} = x \cdot (\mathbb{1}(c = y) - q(c|x, \theta))
\end{align}

Let's compute the gradient for $w_0$, i.e., the weights for the label 0:

In [181]:
c = 0
q0 = q[0]
gradient0 = x * (int(label == answer) - q0)

Of course, we can compute the gradient with pytorch as well. Once we call the method `backward()` in a tensor, all tensors that are used to compute it get an attribute `grad`.

In [180]:
loss.backward()
w.grad

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

Let's check if pytorch gradients match ours. Again, we use the mean squared error instead of the simple `==` operator because of possible differences in precision.

In [182]:
gradient0_pytorch = w.grad[:, 0]
torch.sum((gradient0 - gradient0_pytorch) ** 2)

tensor(1.0068e-17, grad_fn=<SumBackward0>)

That's great! Now we have to effectively change the weights in the direction of the gradients. While we are at it, let's also compute the gradient with respect to the bias.

In [207]:
w.data.sub_(w.grad.data)
b.data.sub_(b.grad.data)

tensor([ 0.0657, -0.0625, -0.0559,  2.3422,  0.1617, -0.0708, -2.1116, -0.6924,
         0.7376,  1.2769])

Run the forward pass again

In [208]:
logits = torch.matmul(x0, w) + b
logits = logits.view([1, -1])
torch.softmax(logits, dim=1)

And the loss:

In [210]:
torch.nn.functional.cross_entropy(logits.view([1, -1]), label.view([1]))

tensor(0., grad_fn=<NllLossBackward>)

We zeroed the loss! So is it that simple to get 100% accuracy classifiers?